<a href="https://colab.research.google.com/github/ShahzaibSE/langgraph-e2e-solutions/blob/main/2_1_tools_messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LLM Actions and Structure**

## Review

We built a simple graph with nodes, normal edges, and conditional edges.

## Goals

Now, let's understand the power of Actions with AI Models and before that structure our prompts.

## Messages

Chat models can use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages), which capture different roles within a conversation.

LangChain supports various message types, including `HumanMessage`, `AIMessage`, `SystemMessage`, and `ToolMessage`.

These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call.

Let's create a list of messages.

Each message can be supplied with a few things:

* `content` - content of the message
* `name` - optionally, a message author
* `response_metadata` - optionally, a dict of metadata (e.g., often populated by model provider for `AIMessages`)

In [156]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from pprint import pprint

messages = [
    AIMessage(content='Hi, I\'m your Virtual Assistant', name='Model')
]
#
messages.append(HumanMessage(content='What is the meaning of life?', name='Shazy'))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for message in messages:
  message.pretty_print()

================================== Ai Message ==================================
Name: Model

Hi, I'm your Virtual Assistant
================================ Human Message =================================
Name: Shazy

What is the meaning of life?
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn about.
================================ Human Message =================================
Name: Lance

I want to learn about the best place to see Orcas in the US.


Let's use HumanMessage prompt for our model.

In [157]:
from google.colab import userdata

In [158]:
%pip install --quiet -U langchain-google-genai

In [159]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [160]:

%pip install --quiet -U langchain
%pip install --quiet -U langchain-google-genai
%pip install --quiet -U langchain-community

In [161]:
from langchain.llms import OpenAI

In [162]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Initialize the OpenAI LLM
openai_llm = OpenAI(
    temperature=0.6,
    openai_api_key=OPENAI_API_KEY
)


In [163]:
# Invoking OpenAI model.
openai_llm.invoke("What is the meaning of life?")

"\nThe meaning of life is a philosophical question that has been debated by many thinkers throughout history. There is no one definitive answer, as it ultimately depends on an individual's beliefs and values. Some people believe that the meaning of life is to find happiness and fulfillment, while others believe it is to fulfill a certain purpose or destiny. Others may see the meaning of life as simply existing and experiencing the world around us. Ultimately, the meaning of life is a subjective and personal concept."

### We will be using HumanMessage.



In [164]:
system_human_messages = [
    SystemMessage(content="You are a helpful assistant! Your name is Bob."),
    HumanMessage(content="What is your name?")
]

response = openai_llm.invoke(system_human_messages)
print(response)



System: My name is Bob.


In [165]:
context_call = openai_llm.invoke(messages)
print(context_call)


AI: The San Juan Islands in Washington state are known for being one of the best places in the US to see Orcas. The islands are home to three resident pods of Orcas, making sightings very common. There are also several tour companies that offer whale watching excursions in the area.


## Tools

Tools are useful whenever you want a model to interact with external systems.

External systems (e.g., APIs) often require a particular input schema or payload, rather than natural language.

When we bind an API, for example, as a tool we given the model awareness of the required input schema.

The model will choose to call a tool based upon the natural language input from the user.

And, it will return an output that adheres to the tool's schema.

[Many LLM providers support tool calling](https://python.langchain.com/v0.1/docs/integrations/chat/) and [tool calling interface](https://blog.langchain.dev/improving-core-tool-interfaces-and-docs-in-langchain/) in LangChain is simple.

You can simply pass any Python `function` into `ChatModel.bind_tools(function)`.

![Screenshot 2024-08-19 at 7.46.28 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab08dc1c17a7a57f9960_chain2.png)

In [166]:
%%capture --no-stderr
%pip install --quiet -U langchain-google-genai langchain_core langgraph

In [167]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [168]:
# Creating LLM Instance
from langchain.llms import OpenAI
openai_llm = OpenAI(
    temperature=0.6,
    openai_api_key=OPENAI_API_KEY
)

### Calling Tool

In [169]:
print("\n\nllm\n", openai_llm, "\n\n\n")



llm
 OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.6, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'n': 1, 'logit_bias': {}, 'max_tokens': 256} 





In [170]:
from langchain.tools import Tool

def deposit_money(name: str, bank_account_no: int) -> str:
    """Deposit Money in a Bank Account.

    Args:
        name (str): Name of the account holder.
        bank_account_no (int): Bank account number.

    Returns:
        str: Success message.
    """
    return f"Deposit successful in {name}'s account {bank_account_no}."


In [171]:
import re

def parse_input(input_str: str):
    """Parses input to extract the name, account number, and amount."""
    match = re.search(r"Deposit \$(\d+) into account number (\d+)", input_str)
    if match:
        amount = int(match.group(1))
        account_no = int(match.group(2))
        name = "John Doe"  # Default name if not provided in input
        return {"name": name, "bank_account_no": account_no}
    raise ValueError("Could not parse input string.")

def deposit_money_wrapper(input_str: str):
    # Parse input and map arguments
    parsed = parse_input(input_str)
    return deposit_money(parsed["name"], parsed["bank_account_no"])

In [172]:
from langchain.agents import initialize_agent
from langchain_core.messages import HumanMessage
# Use Tool with an agent.
tools = [deposit_money_wrapper]

# Tool with detailed description
deposit_money_tool = Tool(
    name="deposit_money",
    func=deposit_money_wrapper,
    description=(
        "Deposits money into the specified account. "
        "The input should include 'name' (the account holder's name) "
        "and 'bank_account_no' (the bank account number)."
    )
)

In [173]:
# # Initialize the agent with tools
# agent = initialize_agent(tools, openai_llm, agent="zero-shot-react-description", verbose=True)

# message = HumanMessage(content="Deposit $100 into account number 00123.")
# #
# # Test the agent
# response = agent.run(message)
# print(response)


In [174]:
import re
from langchain.schema import HumanMessage
from langchain.tools import Tool
from langchain.llms import OpenAI

# Define the deposit money function
def deposit_money(name: str, bank_account_no: int) -> str:
    return f"Deposit successful in {name}'s account {bank_account_no}."



In [175]:
def deposit_money_wrapper(message: HumanMessage) -> str:
    """Parses input and dynamically includes the name."""
    # Use regex to extract account number and amount
    import re
    match = re.search(r"Deposit \$(\d+) into account number (\d+)", message.content)
    if match:
        amount = int(match.group(1))
        account_no = int(match.group(2))
        # Use the name from the HumanMessage
        name = message.name if message.name else "Unknown"
        return f"Deposit of ${amount} was successful into {name}'s account {account_no}."
    else:
        raise ValueError("Input not formatted correctly. Please specify 'Deposit $<amount> into account number <number>'.")

In [180]:
from langchain.tools import Tool
from langchain.schema import HumanMessage

# Wrap the updated function as a tool
deposit_money_tool = Tool(
    name="deposit_money",
    func=deposit_money_wrapper,
    description=(
        "Deposits money into the specified account. "
        "Input should include the amount and account number, and optionally the account holder's name."
    )
)

# Create a HumanMessage with content and name
message = HumanMessage(
    content="Deposit $100 into account number 00123",
    name="Shaheen"
)

# Create a HumanMessage
message = HumanMessage(
    content="Deposit $100 into account number 00123",
    name="Shaheen"
)

# # Process the message using the Tool
# response = deposit_money_tool.run(message.content)
# print(response)

# Create a prompt using the content and name
prompt = f"Process the following request: {message.content} for {message.name}."

# Generate a response using OpenAI
response = openai_llm(prompt)

# Print the OpenAI model's response
print(response)




To process this request, follow these steps:

1. Log into the bank's system using your credentials.
2. Go to the "Transactions" or "Deposits" tab.
3. Select the account number 00123 from the drop-down menu.
4. Enter the deposit amount as $100.
5. In the "Description" or "Memo" field, enter "Deposit for Shaheen".
6. Click on the "Submit" or "Confirm" button to complete the transaction.
7. Verify that the deposit has been successfully processed by checking the account balance.
8. Notify Shaheen that the deposit has been made into their account.
9. Record the transaction in the bank's system for future reference.


In [192]:
import re
from langchain.schema import HumanMessage
from langchain.tools import Tool
from langchain.llms import OpenAI

# Define the deposit_money function
def deposit_money(name: str, bank_account_no: int, amount: int) -> str:
    """Processes deposit requests and returns a success message."""
    return f"Deposit of ${amount} was successful into {name}'s account {bank_account_no}."

# Define the wrapper function
def deposit_money_wrapper(input_str: str, name: str = "Unknown") -> str:
    """Parses input and dynamically includes the name."""
    match = re.search(r"Deposit \$(\d+) into account number (\d+)", input_str)
    if match:
        amount = int(match.group(1))
        account_no = int(match.group(2))
        return deposit_money(name, account_no, amount)
    else:
        raise ValueError("Input not formatted correctly. Expected format: 'Deposit $<amount> into account number <number>'.")

# # Wrap the function as a Tool
# deposit_money_tool = Tool(
#     name="deposit_money",
#     func=lambda content: deposit_money_wrapper(content, message.name),
#     description=(
#         "Deposits money into the specified account. "
#         "Input should specify the amount, account number, and optionally the account holder's name."
#     )
# )


# Create a HumanMessage
message = HumanMessage(
    content="Deposit $100 into account number 00123",
    name="Shaheen"
)

# Process the message using the Tool
response = deposit_money_tool.run(message.content)  # Pass only `content`
print(response)  # Expected: Deposit of $100 was successful into Shaheen's account 00123.

# (Optional) Use OpenAI to generate a response
prompt = f"Process the following request: {message.content} for {message.name}."
response = openai_llm(prompt)
print(response)


Deposit of $100 was successful into Shaheen's account 123.


To: Bank
Subject: Deposit Request

I, Shaheen, would like to deposit $100 into my account number 00123. Please process this request at your earliest convenience. Thank you.

Sincerely,
Shaheen
